In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pickle

In [23]:
train_datapath = "../input/vegetable-image-dataset/Vegetable Images/train"
validation_datapath =  "../input/vegetable-image-dataset/Vegetable Images/validation"
test_datapath = "../input/vegetable-image-dataset/Vegetable Images/test"
labels = {}
enum = 0
for item in sorted(os.listdir(train_datapath)):
    labels[enum] = item
    enum+=1
labels

In [24]:
DataGen = ImageDataGenerator(
    rescale = 1./255
)
train_datagen = DataGen.flow_from_directory(
    train_datapath,
    batch_size = 32,
    class_mode = "categorical")
validation_datagen = DataGen.flow_from_directory(
    validation_datapath,
    batch_size = 32,
    class_mode = "categorical"
)


In [25]:
print(train_datagen.class_indices)
print(validation_datagen.class_indices)

In [26]:
print("Training Images:",train_datagen.samples)
print("Validation Images:",validation_datagen.samples)

In [28]:
from tensorflow.keras.applications import DenseNet169
densenet = DenseNet169(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

In [29]:
for layers in densenet.layers:
    layers.trainable = False
base_output = GlobalAveragePooling2D()(densenet.output)
final_output = Dense(15, activation = "softmax")(base_output)
final_output

In [30]:
my_densenet_model = Model(inputs = densenet.input, outputs = final_output)

In [32]:
early_stopping = EarlyStopping(monitor = "val_loss", patience=5)
model_save = ModelCheckpoint("densenet169_model.h5", save_best_only=True, verbose=1)

In [33]:
my_densenet_model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
history_densenet169 = my_densenet_model.fit(train_datagen, validation_data=validation_datagen, epochs=50, 
             steps_per_epoch = train_datagen.samples//32,
             callbacks=[early_stopping, model_save])

In [45]:
test_data_gen = DataGen.flow_from_directory(
    test_datapath,
    batch_size = 32,
    class_mode = "categorical"
)

In [46]:
my_densenet_model.evaluate(test_data_gen)

In [50]:
with open('./trainHistoryDensenet169', 'wb') as file_pi:
    pickle.dump(history_densenet169.history, file_pi)

In [48]:
image_path = "../input/vegetable-image-dataset/Vegetable Images/train/Broccoli/0003.jpg"
test_image_path = (img_to_array(load_img(image_path, target_size = (224,224)))/255.).reshape(1,224,224,3)

In [49]:
results = np.argmax((my_densenet_model(test_image_path)).numpy(), axis = -1)
labels[int(results)]